# Setup Database, Schema and Stages in Snowflake

## Prerequisite

Install prerequisite libraries by clicking on `Packages` drop-down button at the top menu and under the Anaconda Packages tab make sure the following libraries are installed, if not then you can enter them into the text box and click the `Save` button to install:
- `modin`
- `snowflake-snowpark-python`


## 1. Create database and schema

In [ ]:
-- Create avalanche_db database
CREATE DATABASE IF NOT EXISTS avalanche_db;

-- Create avalanche_schema schema
CREATE SCHEMA IF NOT EXISTS avalanche_schema;

### Determine current database and schema
In case that we have already created both the `AVALANCHE_DB` database and `AVALANCHE_SCHEMA` schema, we can set these as our working database and schema, otherwise it will default to the database and schema that the notebook is running on.

In [ ]:
-- Current database that we are using
SELECT CURRENT_DATABASE();

In [ ]:
-- Current schema that we are using
SELECT CURRENT_SCHEMA();

### Set database and schema

As we expected, if the database and schema have already been created then we are indeed using the same database and schema that the notebook is running on.

In [ ]:
-- Set database to AVALANCHE_DB
USE DATABASE avalanche_db;

-- Set schema to AVALANCHE_SCHEMA
USE SCHEMA avalanche_schema;

## 2. Create stage and load CSV data



### Option 1: Create stage from an S3 bucket

To create a stage we're using `CREATE STAGE` and we're specifying the `URL` as a path to the S3 bucket which points to `'s3://sfquickstarts/misc/avalanche/csv/'`.

In [ ]:
-- Create the stage and load CSV files from S3
CREATE STAGE IF NOT EXISTS avalanche_stage
  URL = 's3://sfquickstarts/misc/avalanche/csv/'
  DIRECTORY = (ENABLE = TRUE AUTO_REFRESH = TRUE);

Let's now check whether these CSV files are loaded properly into the stage.

In [ ]:
-- List files in the stage
ls @avalanche_stage

### Option 2: Create an empty stage, then load files in later



#### Create an empty stage

If we prefer to first create an empty stage and then load in the data separately you can follow instructions herein.

In creating the stage, we're also specifying that the stage be encrypted on the server-side (SSE) and also enabling `DIRECTORY` so that we can see the list of files from the stage.

In [ ]:
CREATE STAGE IF NOT EXISTS avalanche_stage -- Use this if you did not run Option 1
-- CREATE OR REPLACE STAGE avalanche_stage -- Uncomment this if you ran Option 1 (we need to overwrite the previous stage)
  ENCRYPTION = (TYPE = 'SNOWFLAKE_SSE')
  DIRECTORY = (ENABLE = true);

#### Load CSV data into the stage
Download [customer_reviews.csv](https://github.com/https-deeplearning-ai/fast-prototyping-of-genai-apps-with-streamlit/blob/main/M1/Lesson_03/deploy/customer_reviews.csv) from the GitHub repo.

Go to Snowsight then ...# 
1. Click on the Data icon in the left sidebar and select `Database Explorer`
2. Navigate to `AVALANCHE_DB` database > `AVALANCHE_SCHEMA` schema > `Stages` > `AVALANCHE_STAGE`
3. Click on the blue `+ Files` button then in the `Upload Your Files` modal window, click on Browse file or drag and drop the CSV file into the designated area.

## 3. Create a DataFrame from staged CSV file

In [ ]:
import modin.pandas as pd
import snowflake.snowpark.modin.plugin

df = pd.read_csv('@AVALANCHE_STAGE/customer_reviews.csv')
df

## 4. Write to a Snowflake table

Here, we're writing to a Snowflake table called `customer_reviews`

In [ ]:
df.to_snowflake(
    "customer_reviews",
    if_exists="replace",
    index=False
)

## 5. Query table

In [ ]:
SELECT * FROM AVALANCHE_DB.AVALANCHE_SCHEMA.CUSTOMER_REVIEWS;